## PSO 2
The goal of this notebook is to implement the PSO algorithm for hypercubes and simplices based on the Walsh implementation but using the new Designs and Models modules for design initialization and model expansion.

In [31]:
include("../../src/industrial_stats.jl")
using .IndustrialStats: Designs, Models, OptimalityCriteria, TensorOps, Simplex
using Distributions
using LinearAlgebra

In [32]:
include("./types.jl")
using .PSOTypes

## Constants

In [33]:
N = 7
K = 3
S = 4
max_iter = 10
w = 1/(2*log(2))
c1 = 0.5 + log(2)
c2 = 0.5 + log(2)
nn = 3
relTol = 0
maxStag = 500
v_max_scale = 2
max_particle_step_size = Simplex.max_d(K) / v_max_scale

3.7600240536254295

## State

## State Update

In [34]:
function update_memory(memory::ParticleMemory, fitness::ParticleFitness, state::ParticleState)::ParticleMemory
    # Update best positions
    memory.particle_best[fitness.scores .< memory.particle_best_scores, :, :] .= state.particles[fitness.scores .< memory.particle_best_scores, :, :]

    # Update best scores
    memory.particle_best_scores[fitness.scores .< memory.particle_best_scores] .= fitness.scores[fitness.scores .< memory.particle_best_scores]

    # # Update global best score and position
    new_global_best = TensorOps.squeeze(memory.particle_best[argmin(memory.particle_best_scores), :, :])
    new_best_score = minimum(memory.particle_best_scores)

    return ParticleMemory(memory.particle_best, memory.particle_best_scores, new_global_best, new_best_score)
end

function update_fitness(state::ParticleState, objective::Function)::ParticleFitness
    # Compute scores from updated positions
    scores = objective(state.particles)

    # Update fitness
    fitness = ParticleFitness(scores, world.fitness.objective)
    return fitness
end

function update_velocity_and_position(world::ParticleWorld)::ParticleState
    # Get scaling coefficients
    r1 = rand(size(world.state.particles)...)
    r2 = rand(size(world.state.particles)...)

    # Update velocities with inertia, cognitive and social components
    particle_inertia = world.params.w * world.state.velocities
    cognitive = world.params.c1 * r1 .* (world.memory.particle_best .- world.state.particles)
    social = world.params.c2 * r2 .* (TensorOps.expand(world.memory.global_best) .- world.state.particles)

    # Set new velocity
    new_velocities = particle_inertia .+ cognitive .+ social

    # Update particle positions
    new_particle_pos = world.state.particles .+ new_velocities

    return ParticleState(new_particle_pos, new_velocities)
end

function assign_random_neighbors(num_particles, num_neighbors)
    # Create a boolean adjacency matrix of particles
    # Each neighbor has on average num_neighbors connections
    d = Uniform(0, 1)
    p_avg = 1 - (1 - (1 / num_particles)) ^ num_neighbors
    adjacency_matrix = reshape(rand(d, num_particles * num_particles) .< p_avg, (num_particles, num_particles))

    # Connect particles to themselves
    adjacency_matrix[diagind(adjacency_matrix)] .= 1

    return adjacency_matrix
end

function update_neighbors(world::ParticleWorld)::ParticleWorld
    neighbs = assign_random_neighbors(size(world.state.particles, 1), world.params.num_neighbors)
    return ParticleWorld(world.state, neighbs, world.fitness, world.memory, world.params)
end

function update_state(world::ParticleWorld) 
    new_state = update_velocity_and_position(world)
    new_fitness = update_fitness(new_state, world.fitness.objective)
    new_memory = update_memory(world.memory, new_fitness, new_state)
    new_world = ParticleWorld(new_state, world.neighbors, new_fitness, new_memory, world.params)
    return new_world
end

update_state (generic function with 1 method)

## Initialization

In [35]:
function initialize_swarm(initializer::Function, objective::Function, params::HyperParams; num_particles = 150)
    particles = initializer(num_particles)
    velocities = initializer(num_particles)
    particle_state = ParticleState(particles, velocities)
    neighbors = assign_random_neighbors(num_particles, params.num_neighbors)
    scores = objective(particles)
    memory = ParticleMemory(particles, scores, particles[argmin(scores), :, :], minimum(scores))
    fitness = ParticleFitness(scores, objective)
    return ParticleWorld(
        particle_state, 
        neighbors, 
        fitness,
        memory,
        params
    )
end

initialize_swarm (generic function with 1 method)

## Convergence and Termination

In [36]:
function initialize_runner(world::ParticleWorld)
    return RunnerState(world, 0, 0)
end

function should_continue(runner::RunnerState, params::RunnerParams)
    return runner.iter < params.max_iter && runner.stagnation < params.max_stag
end

function improvement(old_world::ParticleWorld, new_world::ParticleWorld, rel_tol)
    return abs(new_world.memory.global_best_score - old_world.memory.global_best_score) > rel_tol
end

function run_pso(world::ParticleWorld, runner_params::RunnerParams, cb::Function)::RunnerState
    runner_state = initialize_runner(world)
    while should_continue(runner_state, runner_params)
        new_world = update_state(runner_state.world)
        if improvement(runner_state.world, new_world, relTol)
            runner_state = RunnerState(new_world, runner_state.iter + 1, 0)
        else
            # Update neighbors
            new_world = update_neighbors(new_world)
            runner_state = RunnerState(new_world, runner_state.iter + 1, runner_state.stagnation + 1)
        end
        cb(runner_state)
    end
    return runner_state
end

run_pso (generic function with 2 methods)

## Testing
### Initialization

In [37]:
# Initialization
initializer = (n) -> rand(n, N, K)
objective = (x) -> TensorOps.squeeze(maximum(x, dims=(2,3)))
params = HyperParams(w, c1, c2, nn)
world = initialize_swarm(initializer, objective, params; num_particles=S)

ParticleWorld(ParticleState([0.9571083012246071 0.7354706018716142 … 0.21565001352435786 0.9363559849910666; 0.6635804464027526 0.9350182479498145 … 0.7278250459852221 0.2285475050586172; 0.6258772161074047 0.1533301354821871 … 0.3516660909471503 0.8336283422610432; 0.4367275035710656 0.4794954851476563 … 0.10821358789310465 0.12517534368130712;;; 0.5263338257464973 0.18158808411301974 … 0.04882032713184459 0.3311530741249008; 0.0384944048809146 0.2253736261726842 … 0.06914201800568631 0.9389175148188372; 0.14590251465105075 0.6317946669959141 … 0.9032832366301288 0.5218933295249694; 0.2898146184070961 0.051287102958875286 … 0.733533481606396 0.6651843798966403;;; 0.8770311412023725 0.8990338134725263 … 0.9107639287011996 0.7668130777666022; 0.200918539260036 0.6030819440336892 … 0.364185306468088 0.5330470993226015; 0.2212237599270116 0.6018098989627217 … 0.021262534034275138 0.0926427530732421; 0.41055374649198684 0.963706854240026 … 0.22041920396666215 0.2515354560979396], [0.071711

### State Update
#### Velocity and Position

In [38]:
# Get scaling coefficients
r1 = rand(size(world.state.particles)...)
r2 = rand(size(world.state.particles)...)

# Update velocities with inertia, cognitive and social components
particle_inertia = world.params.w * world.state.velocities
cognitive = world.params.c1 * r1 .* (world.memory.particle_best .- world.state.particles)
social = world.params.c2 * r2 .* (TensorOps.expand(world.memory.global_best) .- world.state.particles)

# Set new velocity
new_velocities = particle_inertia .+ cognitive .+ social

# Update particle positions
new_particle_pos = world.state.particles .+ new_velocities

new_state = ParticleState(new_particle_pos, new_velocities)

ParticleState([0.6842097675926988 1.4654113711246943 … 0.777635501094562 0.7007390767707404; 1.0750318639141163 1.5317086182216706 … 1.0373429304467028 0.9385565916309617; 1.0832869259776983 1.3592880073722502 … 0.8419965123640121 1.5057795717148132; 0.600263230266513 1.0403482845472907 … 1.0133577144096542 0.8853531178224862;;; 0.6187639762935908 0.6053859615457984 … 0.24206858952006985 0.703283115131321; 0.34319146130875855 0.4121173892279614 … 0.6768861437150854 1.4396261833932122; 0.6810391498627011 0.6970447715713588 … 0.5826144705463234 0.8384436126419996; 0.1033037355358157 0.2830238761517879 … 0.7917704153806095 1.0312436447521371;;; 0.70137914678511 0.747825985621587 … 1.2140712438094428 0.5253366024401024; 0.46348729327990895 1.2499066002903358 … 0.6347688346502383 0.7583900413294871; 0.7875228530651754 0.934719166101837 … 0.5429917195806284 0.38117904229411087; 0.45928359663537843 0.991647463688428 … 0.6925461416472405 0.711855448109378], [-0.27289853363190836 0.729940769253

### Final State

In [39]:
new_world = update_state(world)

ParticleWorld(ParticleState([0.8412382303782885 1.5887287696021155 … 0.8122756288493549 1.0702364421714623; 1.0750318639141163 1.5317086182216706 … 1.0373429304467028 0.9385565916309617; 1.065907181992923 1.2748381279171639 … 0.6586351871913311 0.9756974829113041; 0.5418819400132084 1.1188136966736624 … 0.7270535460672347 0.8392625394432441;;; 0.5306555162055806 0.631266917560751 … 0.24371075729314734 0.8793793243016028; 0.34319146130875855 0.4121173892279614 … 0.6768861437150854 1.4396261833932122; 0.7161027115926575 0.4124751971195052 … 0.7264746383118678 0.7712151274063977; 0.21076813723842822 0.41666468871550544 … 0.627081331062378 0.7776907218388569;;; 0.24350022727418663 0.9046912890503255 … 1.1338245138935794 0.5517501600684431; 0.46348729327990895 1.2499066002903358 … 0.6347688346502383 0.7583900413294871; 0.7831345107872859 0.9340902921890377 … 0.461048631491187 0.5653505855437413; 0.470012462953425 1.0526845742574673 … 0.5736192233453493 0.6265623782794245], [-0.1158700708463

### Runner

In [40]:
# Initialization
initializer = (n) -> rand(n, N, K)
objective = (x) -> TensorOps.squeeze(maximum(x, dims=(2,3)))
params = HyperParams(w, c1, c2, nn)
world = initialize_swarm(initializer, objective, params; num_particles=S)
cb = (x) -> println("Iteration: ", x.iter, ", Stagnation: ", x.stagnation, ", Best score: ", x.world.memory.global_best_score)

#47 (generic function with 1 method)

In [41]:
params = RunnerParams(max_iter, maxStag, relTol)
runner_state = run_pso(world, params, cb)

Iteration: 1, Stagnation: 1, Best score: 0.9155006079842962
Iteration: 2, Stagnation: 2, Best score: 0.9155006079842962
Iteration: 3, Stagnation: 0, Best score: 0.7101011190834949
Iteration: 4, Stagnation: 0, Best score: 0.5969727384872391
Iteration: 5, Stagnation: 0, Best score: 0.34832233378629146
Iteration: 6, Stagnation: 0, Best score: 0.3021134681314523
Iteration: 7, Stagnation: 0, Best score: 0.23015708798937107
Iteration: 8, Stagnation: 1, Best score: 0.23015708798937107
Iteration: 9, Stagnation: 0, Best score: 0.17338379356509864
Iteration: 10, Stagnation: 0, Best score: 0.14586335167810038


RunnerState(ParticleWorld(ParticleState([-0.6451622243684789 -0.061444368774980926 … 0.022345263457058916 0.07912527363485802; -1.5940824646519962 -0.08148583827259914 … -0.018947662517224687 0.10010232055715773; -1.9441617245572802 -0.12544967480742444 … -0.21450076055942643 0.1604566628666984; -2.0502879279119384 -0.11335642699042098 … 0.036608459140062916 0.14586335167810038;;; -0.3062085710105822 -0.06499007417828875 … -0.8129102050572341 -0.15198403070727057; -0.6705746772930485 0.13147147737913506 … -1.3087661635120207 -0.14529217570455816; -0.4104742108398653 0.11501977176235839 … -1.151129481800671 -0.2231705478361614; -0.4468435417407656 -0.032329165293605674 … -1.1608820377021154 -0.32049185988643736;;; -1.1957679261581005 0.5030284249589991 … -0.32663102807617406 -1.4630418068957065; -1.263168299895339 0.16703297946787551 … -0.66956712687781 -1.9077025058265036; -1.4133669676756435 0.08997264931739134 … -0.7553008880955916 -1.5363874324385751; -1.21674339407866 0.12976478150

In [ ]:
runner_state.iter, runner_state.world.memory.global_best_score

(10, 0.14586335167810038)

In [43]:
runner_state.world.memory.global_best

7×3 Matrix{Float64}:
 -2.05029    -0.446844   -1.21674
 -0.113356   -0.0323292   0.129765
 -0.506585   -0.830547    0.0578764
 -1.17671    -0.549901    0.0721687
 -1.45123     0.0434918  -0.950223
  0.0366085  -1.16088    -0.942087
  0.145863   -0.320492   -1.76143

## Mixture